## **Ejercicio 8.02**

Consiste en la elaboración de un informe de Power BI, aprovechando las capacidades analíticas de Python. Se utilizarán los scripts de Python creados previamente en la Tarea 1 para generar visualizaciones personalizadas con las bibliotecas Seaborn y Matplotlib. Estas visualizaciones estarán integradas en el informe de Power BI para ofrecer una comprensión más profunda de la capacidad del lenguaje de programación en la herramienta Power BI.

### **Plan de trabajo para la elaboración del informe en Power BI**

### **Informe de Power BI con Integración de Visualizaciones de Python**

**1. Importar Datos en Power BI**    

Primero, abro Power BI. Luego, utilizo la opción "Obtener datos" para importar los datos que usé en mis scripts de Python.

**2. Conexión a la base de datos MySQL y obtención de datos**       

El siguiente script de Python se conecta a la base de datos MySQL 'TRADES4', obtiene los nombres de las tablas y almacena cada tabla en un DataFrame de pandas.


```python
    import mysql.connector
    import pandas as pd

    # CREO UNA CONEXIÓN PARA LA BASE DE DATOS 'TRADES4'
    cnx = mysql.connector.connect(
        user='root',
        password='250301',
        host='localhost',
        database='trades4')
    cursor = cnx.cursor()

    # Obtener nombres de las tablas y las almaceno en una lista
    cursor.execute("SHOW TABLES")
    table_names = [table_name for (table_name,) in cursor.fetchall()] 
    print(table_names)

    # Obtener DataFrames de las tablas
    dfs = {}
    for table_name in table_names:
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()
        df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
        dfs[table_name] = df

    # Crear variables globales para cada DataFrame
    for table_name, df in dfs.items():
        globals()[f"df_{table_name}"] = df

    # Cerrar cursor y conexión
    cursor.close()
    cnx.close()
```

**3. Establezco las relaciones**        

Una vez que escribo el codigo, entonces lo que hago es crear el esquema para establecer las relaciones. 

El esquema es ![801](files_sprint8/801.png).

**4 Añadir Scripts de Python en Power BI**       

Power BI permite ejecutar scripts de Python directamente. Para hacerlo, voy a la pestaña "Visualizaciones" y selecciono el ícono de Python visual. Después, copio y pego el script de Python directamente en el editor de Power BI.

![802](files_sprint8/802.png).

**5. Seleccionar**      

Esto garantiza que Power BI maneje los datos adecuadamente y que el gráfico muestre el número real de transacciones aceptadas y rechazadas, en lugar de simplemente la frecuencia.      

![803](files_sprint8/803.png)


**6. Cambiar nombre de la  tabla por `dataset`**

Al cambiar el nombre de la tabla a `dataset`, facilito que el script funcione sin problemas y refleje correctamente la información necesaria. Sin esta selección, el gráfico no representará correctamente el número de transacciones.

# Ejercicio 1: Una variable numérica

#### **Código**
---
```python
#### IMPORTAR LAS LIBRERIAS NECESARIAS

import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

#### Convertir 'birth_date' a formato de fecha y calcular la edad en años

dataset['birth_date'] = pd.to_datetime(dataset['birth_date'], format='%b %d, %Y')
dataset['age'] = datetime.now().year - dataset['birth_date'].dt.year

#### Función para crear un boxplot con título
def age_boxplot(df, column):
    sns.set()
    plt.figure(figsize=(8, 4))
    
    #### Crear el boxplot
    ax = sns.boxplot(y=df[column], width=0.2, fliersize=5, linewidth=2.5)
    ax.set_title(f'N1E1 - Boxplot de {column}', fontsize=12, fontweight='bold', pad=10)  # Título modificado
    ax.set_ylabel(f'{column.capitalize()} (años)', fontsize=12, labelpad=10)
    ax.grid(True, linestyle='--', linewidth=0.7)
    ax.yaxis.set_tick_params(labelsize=10)
    sns.despine(top=True, right=True)
    
    #### Anotar cuartiles a la derecha
    quartiles = df[column].quantile([0.25, 0.5, 0.75])
    for q, value in quartiles.items():
        label = f'{int(q * 100)}%: {value:.1f}'
        ax.text(0.125, value, label)
    
    #### Anotar mínimo, máximo y promedio a la izquierda
    stats = {
        'Edad mínima': df[column].min(),
        'Edad máxima': df[column].max(),
        'Edad Promedio': df[column].mean()
    }
    
    #### Calcular posición vertical para los textos
    max_value = max(df[column].max(), max(quartiles))
    min_value = min(df[column].min(), min(quartiles))
    vertical_position = max_value + 0.1 * (max_value - min_value)
    
    for i, (stat, value) in enumerate(stats.items(), start=1):
        ax.text(-0.5, vertical_position - i * 1.2, f'{stat}: {value:.1f}', 
        verticalalignment='top', size='medium', color='black', weight='semibold')
    
    plt.savefig('Age_Boxplot.png')  # Guardar el gráfico como imagen
    plt.close()  # Cerrar el gráfico para liberar recursos

#### Llamar a la función de boxplot con el DataFrame y la columna específica
age_boxplot(dataset, 'age')
```
---

![811](files_sprint8/811.png)

# Ejercicio 2: Dos variables numéricas

#### **Código**
---
```python
    # IMPORTAR LAS LIBRERIAS NECESARIAS
    import pandas as pd
    from datetime import datetime
    import matplotlib.pyplot as plt

    # CONVERTIR 'BIRTH_DATE' A FORMATO DE FECHA Y CALCULAR LA EDAD EN AÑOS
    dataset['birth_date'] = pd.to_datetime(dataset['birth_date'], format='%b %d, %Y')
    dataset['age'] = datetime.now().year - dataset['birth_date'].dt.year

    # COMO ES UN 'OBJECT' HAGO LA CONVERSIÓN A 'FLOAT' PARA OBTENER LAS METRICAS:
    dataset['amount'] = dataset['amount'].astype(float)

    # AGRUPAR LOS TRANSACCIONES POR USUARIO Y CALCULAR EL PROMEDIO DE LA CANTIDAD
    grouped_transactions = dataset.groupby('user_id')['amount'].mean().reset_index()

    # UNIR CON LA EDAD DE CADA USUARIO
    grouped_transactions = grouped_transactions.merge(dataset[['id', 'age']], left_on='user_id', right_on='id')
    # AGRUPAR POR EDAD Y CALCULAR EL PROMEDIO DE LA CANTIDAD
    grouped_by_age = grouped_transactions.groupby('age')['amount'].mean().reset_index()

    # CALCULAR EL PROMEDIO GLOBAL DE AMOUNT
    global_average = grouped_transactions['amount'].mean()

    # CALCULAR CORRELACIÓN DE PEARSON ENTRE EDAD Y PROMEDIO DE COMPRA
    correlation = grouped_by_age['age'].corr(grouped_by_age['amount'])

    # CREAR EL GRÁFICO DE DISPERSIÓN
    plt.figure(figsize=(8, 6))
    plt.scatter(grouped_by_age['age'], grouped_by_age['amount'], marker='o', color='blue', alpha=0.7)

    # AÑADIR LA LÍNEA DEL PROMEDIO GLOBAL
    plt.axhline(y=global_average, color='red', linestyle='--', linewidth=1, label=f'Promedio Global: {global_average:.2f}€')

    # CONFIGURAR TÍTULO Y ETIQUETAS DE EJES
    plt.title('Promedio de Compra por Edad', fontsize=10)
    plt.xlabel('Edad (años)', fontsize=8)
    plt.ylabel('Promedio de Compra (€)', fontsize=10)

    # AJUSTAR LOS TICKS DEL EJE X PARA INCLUIR TODAS LAS EDADES
    plt.xticks(grouped_by_age['age'])

    # AGREGAR LA CORRELACIÓN DE PEARSON DENTRO DEL GRÁFICO
    plt.text(0.05, 0.95, f'Correlación de Pearson: {correlation:.2f}', transform=plt.gca().transAxes,
            fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', edgecolor='black', facecolor='white'))

    # AÑADIR LEYENDA
    plt.legend()

    # MOSTRAR EL GRÁFICO
    plt.grid(True, linestyle='--', linewidth=0.7)
    plt.show()
```
---

![812](files_sprint8/812.png)

# Ejercicio 3: Una variable categórica

Para asegurarme de que el código se ejecute correctamente y se muestren los números de transacciones en lugar de solo la frecuencia, es crucial seleccionar tanto `df_transactions['declined']` como `df_transactions['id']`. 




#### **Código**
---
```python
    import pandas as pd

    dataset = dataset.drop_duplicates()

    # IMPORTAR LAS LIBRERIAS NECESARIAS
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Convertir la columna 'declined' a una variable categórica
    dataset['declined'] = dataset['declined'].astype('category')

    # Crear el gráfico de barras
    plt.figure(figsize=(10, 8))
    sns.countplot(x='declined', data=dataset, palette='viridis')
    plt.title('Número de Transacciones Aceptadas y Rechazadas', fontsize=14)
    plt.xlabel('Estado de Transacción', fontsize=12)
    plt.ylabel('Número de Transacciones', fontsize=12)
    plt.xticks(ticks=[0, 1], labels=['Aceptada', 'Rechazada'], fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', linewidth=0.7)
    plt.show()
```
---

![813](files_sprint8/813.png).

# Ejercicio 4: Una variable categórica y una numérica.

#### **Código**
----------------------------------------------------------------
```python
    # El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 
    import pandas as pd

    dataset = dataset.drop_duplicates()

    # IMPORTAR LAS LIBRERIAS NECESARIAS
    import matplotlib.pyplot as plt
    import seaborn as sns

    plt.figure(figsize=(10, 8))

    # Gráfico de caja de la variable numérica 'amount' por la categórica 'declined'

    sns.boxplot(x='declined', y='amount', data=dataset)
    plt.title('Distribución del Monto por Estado de Transacción')
    plt.xlabel('Estado de Transacción')
    plt.ylabel('Monto')
    plt.xticks(ticks=[0, 1], labels=['Aceptada', 'Rechazada'])

    plt.tight_layout()
    plt.show()
```
---

![814](files_sprint8/814.png).

# Ejercicio 5: Dos variables categóricas.

#### **Código**
----------------------------------------------------------------
```python

    dataset = dataset.drop_duplicates()

    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt

    # Convertir 'declined' y 'country' en variables categóricas
    dataset['declined'] = dataset['declined'].astype('category')
    dataset['country'] = dataset['country'].astype('category')

    # Graficar los datos
    plt.figure(figsize=(10, 6))

    # Gráfico de conteo para las variables categóricas 'declined' y 'country'
    sns.countplot(x='country', hue='declined', data=dataset)
    plt.title('Frecuencia de Transacciones')
    plt.xlabel('País del usuario')
    plt.ylabel('Frecuencia')
    plt.legend(title='Declined', labels=['Aceptada', 'Rechazada'])

    plt.tight_layout()
    plt.show()
```
----------------------------------------------------------------

![815](files_sprint8/815.png).

# Ejercicio 6: Tres variables.

#### **Código**
----------------------------------------------------------------
```python
    import pandas as pd

    dataset = dataset.drop_duplicates()

    # IMPORTAR LAS LIBRERIAS NECESARIAS
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Convertir 'declined' y 'country' en variables categóricas
    dataset['declined'] = dataset['declined'].astype('category')
    dataset['country'] = dataset['country'].astype('category')

    # Graficar los datos
    plt.figure(figsize=(12, 8))

    # Gráfico de barras agrupadas con 'amount' por 'country' y 'declined'
    plt.figure(figsize=(12, 8))
    sns.barplot(x='country', y='amount', hue='declined', data=dataset, estimator=sum)
    plt.title('Grafico de estado de transacciones por pais de origen de compañia y monto...')
    plt.xlabel('País de la compañia')
    plt.ylabel('Monto Total de las transacciones')


    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
```
----------------------------------------------------------------

![816](files_sprint8/816.png)

# Ejercicio 7: Graficar un pairplot

#### **Código**
----------------------------------------------------------------
```python
    dataset = dataset.drop_duplicates()

    # Pegue o escriba aquí el código de script:

    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    from datetime import datetime

    # Calcular la edad a partir de la fecha de nacimiento
    dataset['birth_date'] = pd.to_datetime(dataset['birth_date'], format='%b %d, %Y')  
    dataset['age'] = datetime.now().year - dataset['birth_date'].dt.year  

    # Crear un grupo de edades cada 5 años
    dataset['age_group'] = (dataset['age'] // 5) * 5

    # Definir variables categóricas y numéricas
    categorical_columns = ['country','declined']  
    numeric_columns = ['amount', 'age', 'age_group']  

    # Filtrar el DataFrame para incluir solo las columnas relevantes y eliminar filas con NaN
    plot_data = dataset[categorical_columns + numeric_columns].dropna()

    # Imprimir las columnas seleccionadas
    print(plot_data.columns)

    # Crear el pairplot diferenciando por la columna 'declined' y utilizando el grupo de edad
    sns.pairplot(plot_data, hue='declined', vars=['amount', 'age_group'])

    # Personalizar el gráfico para que muestre la edad cada 5 años en el eje x
    plt.xticks(range(0, int(plot_data['age'].max()) + 1, 5))

    plt.show()
```
----------------------------------------------------------------

![817](files_sprint8/817.png)

# **Dashboard Final:**

![804](files_sprint8/804.png)